# Ising-Lenz Ergodicity: Generate Time Power-laws in Kappa for Field variation

    
     (c) 2013, 2014, 2015, 2016, 2025 Süzen
     GPL v3 

This notebook generates power-laws in the form, $\Kappa_{G}(t) \sim C t^{\alpha}$. Using outputs from generate gamma. power-laws are fitted and results are repored with the bias corrected bootstrap confidence intervals. 

The resulting dataset is called `ising1DrateErgodicityPowerlawsTime.rds`. 

$\Omega_{G}(t)$ : Modified TM G-metric, ergodic convergence.  
$\Gamma_{G}(t)$ : Rate of ergodic convergence.  
$\Kappa_{G}(t)$ : Inverse rate of ergodic convergence.  


* Workhorse of the data generation is method `performIS_repeat`. The RDS file is simply 
an object of list, each member is the output of this method, a named list. See return of this method.
Average magnetisations are reported there, 10 repeats
* Parameters are hardcoded in paralell method below.
     * Sizes at Metropolis (`transP=1`)  and Glauber (`transP=2`) Dynamics `N=512, 1024, 1536`.
     * Field ranges `seq(from = 0.5, by = 0.06, length.out = 16)`.
     * `J=1.0 ikBT=1.0`.
     * MC steps `5e6`.
* KS distances for optimal alpha   
  `1e4, 5e4, 1e5, 1.5e5, 2e5, 2.5e5, 3e5, 6e5, 8e5, 1e6, 1.25e6, 1.5e6, 2e6`


In [1]:
rm(list=ls()) 
require("isingLenzMC"); # Developed and tested on v0.2.5 and R v4.5.1
library(parallel)       # Core package

source("../src/power_utilities.R");

Loading required package: isingLenzMC



In [2]:
fit_power_law <- function(accept_time, gamma_, burn_in = 6) {
  # fit power-law given RunSim's result
  t0 <- accept_time
  g0 <- 1.0 / gamma_
  t0 <- t0[2:length(t0)] # avoid initial inf?
  g0 <- g0[2:length(g0)]
  t <- log10(t0)
  g <- log10(g0)
  ix <- which(t > burn_in)
  x <- t[ix]
  y <- g[ix]
  model <- lm(y ~ x)
  alpha <- as.numeric(model$coefficients[2])
  C <- as.numeric(model$coefficients[1])
  r2 <- summary(model)$adj.r.squared
  yhat <- alpha * x + C
  pairlist(
    alpha = alpha, C = C, accept_times_log = x,
    rate_omega_g_log = y, accept_times_log_full = t,
    rate_omega_g_log_full = g,
    rate_omega_g_log_fit = yhat,
    rate_omega_fit = g0[ix],
    accept_times_fit = t0[ix],
    r2adj = r2
  )
}

ks_distance <- function(pfit, xmin) { # xmin is in log10
  z <- sort(pfit$rate_omega_g_log_fit)
  fecdf <- ecdf(z)
  fecdf_values <- fecdf(z)
  theo_cdf <- 1.0 - abs(xmin / z)^(pfit$alpha)
  D <- max(abs(theo_cdf - fecdf_values))
  D
}

select_powerlaw <- function(accept_time, gamma_, burn_in = 6) {
  # Pick minimal D & corresponding powerlaw
  # On the grid xmin_log
  xmin_log <- log10(c(1e4, 5e4, 1e5, 1.5e5, 2e5, 2.5e5, 3e5, 6e5, 8e5, 1e6, 1.25e6, 1.5e6, 2e6))
  lx <- length(xmin_log)
  pfits <- vector("list", lx)
  Ds <- vector("list", lx)
  for (i in 1:lx) {
    pfits[[i]] <- fit_power_law(accept_time, gamma_, burn_in = xmin_log[[i]])
    Ds[[i]] <- ks_distance(pfits[[i]], xmin_log[[i]])
  }
  Ds_numeric <- as.numeric(Ds)
  dix <- which(Ds_numeric == min(Ds_numeric))
  pairlist("Ds" = Ds_numeric[[dix]], "pfit" = pfits[[dix]], "xmin" = xmin_log[[dix]], "naccept" = length(accept_time), "dix" = dix)
}

library(boot)

get_mean_boot95_bca <- function(data) {
  # Bias-corrected bootstraped means and 95$ CI.
  mean_boot <- function(data, indices) {
    return(mean(data[indices]))
  }
  boot_result <- boot(data = as.numeric(data), statistic = mean_boot, R = 1000)
  boot_means <- boot_result$t
  ci <- boot::boot.ci(boot_result, type = "bca")$bca[4:5]
  list("mean" = boot_result$t0, "lower" = ci[1], "upper" = ci[2], "ci" = ci)
}

In [3]:
time_power_law <- function(result) {
    # Compute power-law with bias corrected bootstrap
    nrepeat <- length(result$accept_times_repeats)
    pfit_objects <- vector("list", nrepeat)
    alphas <- vector("list", nrepeat)
    Cs <- vector("list", nrepeat)
    optDs <- vector("list", nrepeat)
    opt_xmins <- vector("list", nrepeat)
    naccepts <- vector("list", nrepeat)
    adjR2s <- vector("list", nrepeat)
    timeslog <- vector("list", nrepeat)
    omegaprimeslogs <- vector("list", nrepeat)

    for (i in 1:nrepeat) {
        t <- result$accept_times_repeats[[i]]
        ga <- result$gamma_repeats[[i]]
        pl <- select_powerlaw(t, ga)
        pfit_objects[[i]] <- pl$pfit
        alphas[[i]] <- pl$pfit$alpha
        Cs[[i]] <- pl$pfit$C
        optDs[[i]] <- pl$D
        opt_xmins[[i]] <- pl$xmin
        naccepts[[i]] <- pl$naccept
        adjR2s[[i]] <- pl$pfit$r2adj
        timeslog[[i]] <- pl$pfit$accept_times_log_full
        omegaprimeslogs[[i]] <- pl$pfit$rate_omega_g_log_full
    }




    # alpha
    alpha_s <- get_mean_boot95_bca(as.numeric(alphas))
    alpha_mean <- alpha_s$mean
    alpha_boot95ci <- c(alpha_s$lower, alpha_s$upper)
    # C
    C_s <- get_mean_boot95_bca(as.numeric(Cs))
    C_mean <- C_s$mean
    C_boot95ci <- c(C_s$lower, C_s$upper)

    # xmin
    xmin_s <- get_mean_boot95_bca(as.numeric(opt_xmins))
    xmin_mean <- xmin_s$mean
    xmin_boot95ci <- c(xmin_s$lower, xmin_s$upper)

    # adjR2
    adjR2_s <- get_mean_boot95_bca(as.numeric(adjR2s))
    adjR2_mean <- adjR2_s$mean
    adjR2_boot95ci <- c(adjR2_s$lower, adjR2_s$upper)

    # optD
    optDs_s <- get_mean_boot95_bca(as.numeric(optDs))
    optDs_mean <- optDs_s$mean
    optDs_boot95ci <- c(optDs_s$lower, optDs_s$upper)


    meta_info <- list(
        "N" = result$N,
        "dynamics" = result$dynamics,
        "ikBT" = result$ikBT,
        "alpha_mean" = alpha_mean,
        "alpha_lower" = alpha_boot95ci[1],
        "alpha_upper" = alpha_boot95ci[2],
        "C_mean" = C_mean,
        "C_lower" = C_boot95ci[1],
        "C_upper" = C_boot95ci[2],
        "adjR2_mean" = adjR2_mean,
        "adjR2_lower" = adjR2_boot95ci[1],
        "adjR2_upper" = adjR2_boot95ci[2],
        "xmin_mean" = xmin_mean,
        "xmin_lower" = xmin_boot95ci[1],
        "xmin_upper" = xmin_boot95ci[2],
        "optDs_mean" = optDs_mean,
        "optDs_lower" = optDs_boot95ci[1],
        "optDs_upper" = optDs_boot95ci[2],
        "H" = result$H,
        "J" = result$J,
        "nrepeat" = nrepeat
    )
    meta_info
}


In [4]:
ergoRates <- readRDS("../datasets/ising1DrateErgodicityFields.rds")

In [5]:
#
# Power-laws in the form: Kappa = C t^alpha
# Generate chucks of 8  for parallel run.
#
# ~20m MacM2

to_null <- function(x) {
    if (is.null(x)) NA else x
}

for(i in 1:12) {
    result_ids <- c(1:8)+(i-1)*8
    get_result <- function(i) time_power_law(ergoRates[[i]])
    par_result <- mclapply(result_ids, get_result, mc.cores = 8)
    for(j in 1:8) {
        if(i == 1 & j == 1) {
            df_gamma <- as.data.frame(par_result[[j]])
        } else {
            df_gamma <- rbind.data.frame(df_gamma, as.data.frame(lapply(par_result[[j]], to_null)))
        }
    }
}


In [6]:
saveRDS(df_gamma, "../datasets/ising1DrateErgodicityPowerlawsTimeFields.rds")